In [1]:
# For this notebook to run with updated APIs, we need torch 1.12+ and torchvision 0.13+
try:
    import torch
    import torchvision
    assert int(torch.__version__.split(".")[1]) >= 12, "torch version should be 1.12+"
    assert int(torchvision.__version__.split(".")[1]) >= 13, "torchvision version should be 0.13+"
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")
except:
    print(f"[INFO] torch/torchvision versions not as required, installing nightly versions.")
    !pip3 install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
    import torch
    import torchvision
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")

[INFO] torch/torchvision versions not as required, installing nightly versions.
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
torch version: 2.9.1+cpu
torchvision version: 0.24.1+cpu


In [22]:
import sys
sys.path.append(r"E:\Projects\0-deeplearning-pytorch\deep-learning-by-pytorch")


In [24]:
from going_modular import data_setup, engine


In [25]:
# Continue with regular imports
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms

# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

# Try to import the going_modular directory, download it from GitHub if it doesn't work
try:
    from going_modular.going_modular import data_setup, engine
except:
    # Get the going_modular scripts
    print("[INFO] Couldn't find going_modular scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .
    !rm -rf pytorch-deep-learning
    from going_modular import data_setup, engine

[INFO] Couldn't find going_modular scripts... downloading them from GitHub.


fatal: destination path 'pytorch-deep-learning' already exists and is not an empty directory.
'mv' is not recognized as an internal or external command,
operable program or batch file.
'rm' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
def set_seed (seed:int=42):
    import random
    import numpy as np
    import torch

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [4]:
import os 
import zipfile
from pathlib import Path 
import requests 
def download_data  (source: str,destination:str,remove_source:bool=True):
    """Download and unzip data from a given URL.

    Args:
        source (str): URL of the zip file to download.
        destination (str): Path to the directory where the data should be extracted.
        remove_source (bool, optional): Whether to delete the zip file after extraction. Defaults to True.
    """
    # Download the file from `source` and save it locally under `destination`:
    data_path=Path("data/")
    image_path=data_path/destination
    if image_path.is_dir():
        print(f"[INFO] {destination} already exists. Skipping download.")
    else:
        print(f"[INFO] Downloading {source} to {destination}...")
        image_path.mkdir(parents=True,exist_ok=True)
        target_file=Path(source).name
        with open (data_path/target_file, "wb")as f:
            request=requests.get(source)
            print(f"[INFO] Downloading {target_file}...")
            f.write(request.content)
        # Unzip the file
        print(f"[INFO] Unzipping {target_file}...")
        with zipfile.ZipFile(data_path/target_file, 'r') as zip_ref:
            zip_ref.extractall(image_path)
    # Remove the zip file
        if remove_source:
            os.remove(data_path/target_file)
            print(f"[INFO] Removed {target_file}.")
            
    return image_path
            
    

In [5]:
image_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                           destination="pizza_steak_sushi")
image_path

[INFO] pizza_steak_sushi already exists. Skipping download.


WindowsPath('data/pizza_steak_sushi')

In [6]:
train_dir=image_path/"train"
test_die= image_path/"test"
train_dir, test_die

(WindowsPath('data/pizza_steak_sushi/train'),
 WindowsPath('data/pizza_steak_sushi/test'))

In [9]:
# set up imagenet normalization values
from torchvision import transforms
normalize = torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                             std=[0.229, 0.224, 0.225])
manual_transforms=transforms.Compose([
    transforms.Resize((224,224)),transforms.ToTensor(),normalize])

In [14]:
import os
os.getcwd()


'e:\\Projects\\0-deeplearning-pytorch\\deep-learning-by-pytorch'

In [30]:
from going_modular import engine,utils,model_builder,data_setup
from torchvision import datasets,models,transforms
from torch.utils.data import DataLoader 
train_dataloader,test_dataloader,class_names=data_setup.create_dataloaders(train_dir=train_dir,
                                                                        test_dir=test_die,transform=manual_transforms,batch_size=32)